# Calour Experiment class tutorial
Learn about how calour stores the data of an experiment

## Setup

In [1]:
import calour as ca
ca.set_log_level(11)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## Load the data
we use the chronic fatigue syndrome data from:

Giloteaux, L., Goodrich, J.K., Walters, W.A., Levine, S.M., Ley, R.E. and Hanson, M.R., 2016.

Reduced diversity and altered composition of the gut microbiome in individuals with myalgic encephalomyelitis/chronic fatigue syndrome.

Microbiome, 4(1), p.30.

In [2]:
cfs=ca.read_amplicon('data/chronic-fatigue-syndrome.biom',
                     'data/chronic-fatigue-syndrome.sample.txt',
                     normalize=10000,min_reads=1000)

2020-09-11 20:15:24 INFO loaded 87 samples, 2129 features
2020-09-11 20:15:24 WARNING These have metadata but do not have data - dropped (1): {'ERR1331814'}
2020-09-11 20:15:24 WARNING Do you forget to normalize your data? It is required before running this function
2020-09-11 20:15:24 INFO After filtering, 87 remain.


## The `Experiment` class
Calour stores the experiment as two Pandas.DataFrame (for sample_metadata and feature_metadata) and a (sparse or dense) data matrix.

The order in the dataframes and the table is synchronized, so entry number X in the sample_metadata dataframe always corresponds to row X in the data matrix (and similarily entry Y in the feature_metadata always corresponds to column Y in the data matrix).

## The \__repr__ of the Experiment
Contains the class (in our case - ca.AmpliconExperiment - which is derived from ca.Experiment),

the original biom table filename,

and how many samples and features does it have.

In [3]:
print(cfs)

AmpliconExperiment with 87 samples, 2129 features


## The per-sample metadata (`Experiment.sample_metadata`)
This is a Pandas.DataFrame, with the index being the SampleID, and columns for the sample metadata fields (loaded from the mapping file).

Note that Calour also added the "_calour_original_abundance" field

In [4]:
cfs.sample_metadata

,BioSample_s,Experiment_s,MBases_l,MBytes_l,Run_s,SRA_Sample_s,Sample_Name_s,Assay_Type_s,AssemblyName_s,BioProject_s,...,Subject,Emotional_well_being,Role_physical,Bell,Physical_functioning,Pain,Age,BMI,_sample_id,_calour_original_abundance
#SampleID,,,,,,,,,,,,,,,,,,,,,
ERR1331798,SAMEA3904128,ERX1403418,43,29,ERR1331798,ERS1091262,LR16,AMPLICON,<not provided>,PRJEB13092,...,Patient,72.0,0.0,20.0,75.0,23.0,50,37.59,ERR1331798,62629.0
ERR1331812,SAMEA3904142,ERX1403432,77,54,ERR1331812,ERS1091276,LR72,AMPLICON,<not provided>,PRJEB13092,...,Patient,56.0,NaN,30.0,60.0,68.0,64,22.85,ERR1331812,96404.0
ERR1331836,SAMEA3904166,ERX1403456,83,56,ERR1331836,ERS1091300,LR42,AMPLICON,<not provided>,PRJEB13092,...,Control,NaN,NaN,NaN,NaN,NaN,35,30.66,ERR1331836,105470.0
ERR1331831,SAMEA3904161,ERX1403451,38,26,ERR1331831,ERS1091295,IC10,AMPLICON,<not provided>,PRJEB13092,...,Control,NaN,NaN,NaN,NaN,NaN,45,22.24,ERR1331831,50560.0
ERR1331815,SAMEA3904145,ERX1403435,49,33,ERR1331815,ERS1091279,LR75,AMPLICON,<not provided>,PRJEB13092,...,Patient,NaN,NaN,NaN,NaN,NaN,41,32.30,ERR1331815,66414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR1331818,SAMEA3904148,ERX1403438,54,38,ERR1331818,ERS1091282,IC04,AMPLICON,<not provided>,PRJEB13092,...,Control,NaN,NaN,NaN,NaN,NaN,32,32.98,ERR1331818,71017.0
ERR1331792,SAMEA3904122,ERX1403412,32,22,ERR1331792,ERS1091256,LR09,AMPLICON,<not provided>,PRJEB13092,...,Control,24.0,100.0,90.0,80.0,65.0,43,22.14,ERR1331792,41598.0
ERR1331857,SAMEA3904187,ERX1403477,61,42,ERR1331857,ERS1091321,IC8,AMPLICON,<not provided>,PRJEB13092,...,Control,NaN,NaN,NaN,NaN,NaN,53,29.02,ERR1331857,89919.0


## The per-feature metadata (`Experiment.feature_metadata`)
This is a Pandas.DataFrame, with the index being the featureID (usually the sequence), and columns for the feature metadata (usually "taxonomy", and also additional fields added by calour following differential abundance testing)

In [5]:
cfs.feature_metadata

,_feature_id,taxonomy
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGACGCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGTGTCTTGAGTACAGTAGAGGCAGGCGGAATTCGTGG,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGGTTGTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGCGACCTTGAGTGCAACAGAGGTAGGCGGAATTCGTGG,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...
TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGCAGGCGGTGCGGCAAGTCTGATGTGAAAGCCCGGGGCTCAACCCCGGGACTGCATTGGAAACTGTCGTACTTGAGTATCGGAGAGGTAAGTGGAATTCCTAG,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGATATCTTGAGTGCAGTTGAGGCAGGCGGAATTCGTGG,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...
TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGTGGATTGTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGAAACTGGCAGTCTTGAGTACAGTAGAGGTGGGCGGAATTCGTGG,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...
...,...,...
TACGTAGGGAGCAAGCGTTGTCCGGATTTACTGGGTGTAAAGGGTGCGTAGGCGGATTGGCAAGTCAGAAGTGAAATCCATGGGCTTAACCCATGAACTGCTTTTGAAACTGTTAGTCTTGAGTGAAGCAGAGGTAGGCGGAATTCCCGG,TACGTAGGGAGCAAGCGTTGTCCGGATTTACTGGGTGTAAAGGGTG...,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCATGTAGGCGGTTCCCTAAGTCGGTCGTGAAAATGCGGTGCTTAACGCCGTATGGCGATCGAAACTGGGGGACTTGAGTGCAGGAGAGGAAAGGGGAACTCCCAGT,TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCA...,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...
TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGTTAATTAAGTTAGTGGTCAAATCCGGAGGCTTCACTTCCGATCGCCATTAAAACTGATTAGCTAGAGAATGGACGAGGTAGGCGGAATAAGTTAA,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTG...,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...
TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCTGTGTAAGTCTGAAGTGAAAGCCCGGGGCTCAACCCCGGGACTGCTTTGGAAACTGTATAGCTAGAGTGCTGGAGAGGTAAGTGGAATTCCTAG,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...


## Reads table (`Experiment.data`)
This is a numpy 2D array or a scipy.Sparse matrix containing the feature X sample reads.

Rows are samples, columns are features.

In [6]:
cfs.data

<87x2129 sparse matrix of type '<class 'numpy.float64'>'
	with 21995 stored elements in Compressed Sparse Row format>

### Choosing sparse/dense representation of the data

When loading the data, it is by default loaded as a scipy.Sparse.CSR matrix (which is more memory efficient for sparse data).

We can force Calour to load the data as a dense numpy 2D array using the `sparse=False` parameter in the `read_amplicon()` function.

We can also convert between sparse and dense using the `sparse` attribute of the experiment

#### Convert to dense

In [7]:
cfs.sparse=False
cfs.data

array([[3.17744176e+03, 9.53232528e+02, 7.34643695e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.59231982e+03, 3.83801502e+00, 3.63055475e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.12373187e+03, 4.78524699e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.01104327e+03, 0.00000000e+00, 2.22422402e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.40518245e+02, 1.77466873e+03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.02440604e+02, 0.00000000e+00, 0.00000000e+00, ...,
        3.39941226e+01, 5.10534442e+00, 1.36972655e+00]])

#### convert to sparse

In [8]:
cfs.sparse=True
cfs.data

<87x2129 sparse matrix of type '<class 'numpy.float64'>'
	with 21995 stored elements in Compressed Sparse Row format>

### Getting the data
We can use the `Experiment.get_data()` function to obtain a copy of the data, either as sparse or dense.

#### sparse=None means keep the original format

In [9]:
dat = cfs.get_data(sparse=None)
dat

<87x2129 sparse matrix of type '<class 'numpy.float64'>'
	with 21995 stored elements in Compressed Sparse Row format>

#### sparse=True returns a sparse representation of the data (copies if needed)

In [10]:
dat = cfs.get_data(sparse=True)
dat

<87x2129 sparse matrix of type '<class 'numpy.float64'>'
	with 21995 stored elements in Compressed Sparse Row format>

#### sparse=False returns a dense representation of the data (copies if needed)

In [11]:
dat = cfs.get_data(sparse=False)
dat

array([[3.17744176e+03, 9.53232528e+02, 7.34643695e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.59231982e+03, 3.83801502e+00, 3.63055475e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.12373187e+03, 4.78524699e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.01104327e+03, 0.00000000e+00, 2.22422402e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.40518245e+02, 1.77466873e+03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.02440604e+02, 0.00000000e+00, 0.00000000e+00, ...,
        3.39941226e+01, 5.10534442e+00, 1.36972655e+00]])

#### we can also force copying the data using `copy=True`

In [12]:
dat = cfs.get_data(sparse=None, copy=False)
dat is cfs.data

True

In [13]:
dat = cfs.get_data(sparse=None, copy=True)
dat is cfs.data

False

#### getting a single entry based on feature and sample values
We can use the \__getitem(sampleid, featureid)\__ attribute.

In [14]:
cfs['ERR1331815','TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGCGGACGCTTAAGTCAGTTGTGAAAGTTTGCGGCTCAACCGTAAAATTGCAGTTGATACTGGGTGTCTTGAGTACAGTAGAGGCAGGCGGAATTCGTGG']

1407.0828439786792

## Saving an Experiment

### Save everything (biom table+sample/feature mapping files)
By default saves as an HDF5 biom table. We can save as a text biom table insteadusing `fmt="txt"` parameter

In [15]:
cfs.save('cfs')

In [16]:
!ls cfs*

cfs.biom  cfs_feature.txt  cfs_sample.txt
